# **6. Modelo Final**

## 6.1. Selección del Mejor Modelo
Tras evaluar distintos modelos (KNN, Árboles de Decisión, Regresión Logística, SVM, etc) se comprobó que el SVM con Kernel RBF y los hiperparámetros óptimos:
* C=10
* gamma = 'scale'
* kernel = 'rbf'

presenta el mejor desempeño en términos de Balanced Accuracy en el conjunto de validación interna. Esto lo convierte en nuestro candidato para el modelo final

## 6.2.Generación de Predicciones para la Competición
El siguiente paso es utilizar el modelo final para generar predicciones sobre el conjunto de datos de la competición. Para ello se debe aplicar el mismo preprocesado que a los datos de entrenamiento. El fichero resultante se guardará como predicciones.csv

In [20]:
import pandas as pd
import joblib
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Importamos los datos de entrenamiento
datos_generales_originales = pd.read_csv('./attrition_availabledata_10.csv.gz')
print("Datos originales de entrenamiento cargados para ajustar el preprocesador.")


# Para el conjunto de competición, eliminamos las mismas columnas que eliminamos en el entrenamiento
datos_generales_originales['Attrition'] = datos_generales_originales['Attrition'].map({'Yes': 1, 'No': 0})
X_entrenamiento = datos_generales_originales.drop(columns=['Attrition', 'EmployeeID', 'Over18', 'EmployeeCount', 'StandardHours'])

# REDEFINIMOS MANUALMENTE EL PREPROCESADOR UTILIZADO EN EL BEST MODEL

categorical_features = ['Department', 'JobRole', 'EducationField']
ordinal_features = ['BusinessTravel', 'Gender', 'MaritalStatus']

valores_ordinales = [
    ['Non-Travel', 'Travel_Rarely', 'Travel_Frequently'],   # BusinessTravel
    ['Male', 'Female'],                                     # Gender
    ['Single', 'Married', 'Divorced']                       # MaritalStatus 
]

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features),
        ('ord', OrdinalEncoder(categories=valores_ordinales, handle_unknown='use_encoded_value', unknown_value=-1), ordinal_features)
    ],
    remainder='passthrough' 
)

# Ajustamos el preprocesador con TODOS los datos de X 
preprocessor.fit(X_entrenamiento)

# --- PASO 2: CargaMOS datos de competición y modelo final ---
datos_test_competicion = pd.read_csv('./attrition_competition_10.csv.gz')
modelo_final = joblib.load('modelo_final_Grupo5.pkl')


columnas_a_eliminar_test = ["EmployeeID", "EmployeeCount", "Over18", "StandardHours"]

# Asegúrate de que datos_test_competicion exista y sea el DataFrame correcto
datos_test_competicion_limpios = datos_test_competicion.drop(columns=columnas_a_eliminar_test)

# No eliminamos 'Attrition' aquí porque no existe en los datos de competición

# Aplicar el preprocesador ajustado a los datos de competición
datos_test_transformados = preprocessor.transform(datos_test_competicion_limpios)

# Convertimos a DataFrame para facilitar la manipulación
columnas_prefijadas = preprocessor.get_feature_names_out()
datos_test_listos_para_predecir = pd.DataFrame(datos_test_transformados, columns=columnas_prefijadas)

# --- PASO 3: Generamos las Predicciones ---
print("\nGenerando predicciones...")

# Pasamos el DataFrame con los nombres de columna CORRECTOS (prefijados)
y_pred_test = modelo_final.predict(datos_test_listos_para_predecir)

# Creamos un DataFrame con las predicciones y el EmployeeID original
df_predicciones = pd.DataFrame({
    'EmployeeID': datos_test_competicion['EmployeeID'], # Usamos el ID original
    'Attrition': y_pred_test # La columna de predicción
})

# Mapeamos de nuevo a 'Yes'/'No' si es necesario para el formato de salida
df_predicciones['Attrition'] = df_predicciones['Attrition'].map({1: 'Yes', 0: 'No'})

print(f"\nDistribución de predicciones:\n{df_predicciones['Attrition'].value_counts()}")

# Guardamos las predicciones
output_filename = "predicciones_Grupo5.csv"
df_predicciones.to_csv(output_filename, index=False)
print(f"\nPredicciones guardadas en '{output_filename}'")


Datos originales de entrenamiento cargados para ajustar el preprocesador.

Generando predicciones...

Distribución de predicciones:
Attrition
No     1236
Yes     234
Name: count, dtype: int64

Predicciones guardadas en 'predicciones_Grupo5.csv'


# **7. Tarea Adicional**


Para esta tarea adicional, hemos decicido utilizar la herramienta SHAP, para interpretar nuestro modelo final.

SHAP (SHapley Additive exPlanations) es una herramienta basada en la teoría de juegos que permite cuantificar la contribución de cada variable a la predicción de un modelo. Entre sus principales ventajas destacan:

* **Interpretabilidad Local y Global**: SHAP ofrece explicaciones tanto a nuvel individual (por cada predicción) como a nivel global (importancia de las variables a lo largo de todo el conjunto). Esto permite identificar cómo cada característica influye en cada decisión del modelo.

* **Consistencia y solidez**: Los valores de Shapley garantizan que la contribución de cada característica se mida de forma justa, lo que otorga una interpretación coherente y consistente.

* **Modelo-agnóstico**: A diferencia de otros métodos que se basan en la estructura interna del modelo (por ejemplo, la importancia basada en la impureza de los árboles en RandomForest), SHAP puede aplicarse a cualquier modelo.

* **Transparencia**: Integrar SHAP en el análisis del modelo permite detectar posibles sesgos o relaciones inesperadas entre variables, lo cual es fundamental para generar confianza en la toma de decisiones basadas en inteligencia artificial.

Realizar esta tarea extra con SHAP resulta especialmente interesante en comparación con construir otro modelo como un RandomForest o una red neuronal. Aunque estos modelos pueden ofrecer buenos resultados, la interpretación de sus resultados (por ejemplo, mediante la importancia de variables basada en el Gini o técnicas de regularización) suele ser menos precisa y profunda. En cambio, al interpretar el modelo SVM seleccionado con SHAP, se obtiene una explicación detallada y cuantitativa de la influencia de cada característica en la predicción, lo cual añade un valor extra al análisis, especialmente en contextos donde la explicabilidad es crucial.

## 7.1 Código para usar SHAP en nuestro modelo final

Primero, debemos instalar la libreria de shap en nuestro entorno virtual

In [21]:
!pip install shap

In [ ]:
import shap
import matplotlib.pyplot as plt
import pandas as pd

#Usamos el DataFrame X_entrenamiento para el preprocesamiento
X_entrenamiento_transformado_np = preprocessor.transform(X_entrenamiento) # Sale como NumPy array

columnas_preprocesadas = preprocessor.get_feature_names_out()

X_entrenamiento_preprocesado = pd.DataFrame(X_entrenamiento_transformado_np, columns=columnas_preprocesadas)

# Obtenemos los nombres de las columnas DESPUÉS del preprocesamiento

# Seleccionar una muestra representativa del conjunto preprocesado para el fondo (background)
# Reducir el tamaño del fondo para acelerar el cálculo
X_background = X_entrenamiento_preprocesado.sample(100, random_state=42)

# Definir la función de predicción asegurándonos de que se pasan los nombres de las columnas
def model_predict_proba(data):
    # KernelExplainer puede pasar un NumPy array, lo convertimos a DataFrame
    # con las columnas correctas ANTES de pasarlo al modelo.
    if not isinstance(data, pd.DataFrame):
        data_df = pd.DataFrame(data, columns=columnas_preprocesadas)
    else:
        data_df = data #
    # Accedemos al mejor estimador y usamos .predict()
    return modelo_final.best_estimator_.predict(data_df)

# Crear el explicador con KernelExplainer usando el fondo reducido
explainer = shap.KernelExplainer(model_predict_proba, X_background)

# Seleccionar un subconjunto reducido de datos para explicar
X_explain = X_entrenamiento_preprocesado.sample(200, random_state=42)

# Calcular los valores SHAP para el subconjunto seleccionado, limitando nsamples para acelerar el proceso
shap_values = explainer.shap_values(X_explain, nsamples=100)

# Visualizar el resumen de los valores SHAP para identificar la importancia de las variables
shap.summary_plot(shap_values, X_explain, feature_names=columnas_preprocesadas)
plt.show()

# Explicación local de una instancia en particular
i = 0  # índice de la instancia a explicar
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[i], X_explain.iloc[i], feature_names=columnas_preprocesadas)



 46%|████▌     | 91/200 [00:41<00:48,  2.23it/s]

Esta representación SHAP nos da una idea de qué aspectos considera más relevantes el modelo SVC al predecir la Attrition (si un trabajador se va o se queda). Aparte, nos deja ver cómo un valor alto o bajo en cada factor afecta a empleados concretos.

Lo más determinante sigue siendo, de manera uniforme, *remainderhrs*. Niveles altos (puntos rojos) elevan de forma notable la opción de marcha (SHAP positivos), mientras que niveles reducidos (azules) la minimizan (SHAP negativos).

Tras este aspecto esencial, vemos un conjunto de variables de importancia continua, aunque su orden justo podría variar un poco entre usos o con distintas muestras. Entre las más destacadas en este estudio están *ord_MaritalStatus* (estar soltero, azul , eleva mucho el riesgo), *remainder_TotalWorkingYears* (más experiencia, rojo, reduce el peligro), *remainder_TrainingTimesLastYear* (más formación, rojo, parece elevar el peligro, un punto curioso para remarcar) y *remainder_JobSatisfaction* (poca satisfacción, azul, eleva el peligro). La variable *ord_BusinessTravel* (viajar mucho, rojo, eleva el peligro) también tiende a salir con una influencia importante.

Otras variables como el tiempo desde el último ascenso (*remainder_YearsSinceLastPromotion*), la satisfacción con el entorno (*remainder_EnvironmentSatisfaction*), formar parte de ciertos departamentos (p. ej. , *cat_Department_Sales*) y la edad (*remainder_Age*) también enseñan influencia relevante en las predicciones del modelo, pese a que su sitio exacto en la lista puede cambiar.

En líneas generales, el análisis SHAP da a entender que, para el modelo SVC, aspectos como las horas de trabajo, el estado civil, el recorrido reunido, la formación reciente, la satisfacción en el trabajo y los viajes son cruciales para anticipar la rotación. Si bien la importancia relativa justa puede presentar pequeñas variaciones, los patrones notados (cómo niveles altos o reducidos de estas variables afectan la predicción) son coherentes y lógicos en su mayoría. Esto nos ayuda a comprender mejor cómo decide el modelo y a fiarnos de sus resultados.